In [1]:
import tweepy
import json
import psycopg2
import data_cred as cred
from geopy.geocoders import Nominatim

In [2]:
consumer_key = cred.api_key
consumer_secret = cred.api_secrect_key
access_token = cred.access_token
access_token_secret = cred.access_token_secrest
geolocator = Nominatim(user_agent="twitter_scrapper")

In [3]:
def api_connection():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
        print("auth ok")
    except Exception as e:
        print(e.Exception)

    return api

In [4]:
api = api_connection()

auth ok


In [12]:
def extract_tweets(hashtag, count, geo):
    #api = api_connection()
    print('Init..')
    tweets_json = []
    tuits_cursor = tweepy.Cursor(api.search,q=hashtag, lang='es', result_type='recent', rpp=100, include_rts=False,tweet_mode='extended',geocode=geo).items(count)
    #a = tuits_cursor.next()
    for tweet in tuits_cursor:
        if (not tweet.retweeted and 'RT @' not in tweet.full_text):
            tweets_json.append(tweet._json)
            #avatar, geo_loc, verificada?, tag, username, user_id, entities(img, url, etc), source(android,ios,web)
    print('Ending..')
    return tweets_json

In [13]:
def apply_filter(topic):
    since_until=' since:2020-10-12 until:2020-10-20'
    filter_retweets=" -filter:retweets"
    topic = topic+since_until
    topic = topic+filter_retweets
    return topic

In [14]:
def connect_db(user,password,host,port,database):
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")              
    return cursor,connection

In [18]:
def search_by_geo(geo_data, topic, count, cursor, connection ):
    for geo in geo_data:
        location = geolocator.reverse(geo)
        geo_t = geo+',400km'
        tweets = extract_tweets(apply_filter(topico_buscar_a),count, geo_t)
        print(location.raw['address']['city'],location.raw['address']['country'], f' topic = {topic},  Count = {len(tweets)}')
        insert_tweets(tweets, topic,cursor,connection,location )
    return tweets


In [19]:
def insert_tweets(tweets, topic,cursor,connection,location):
    for  tweet in tweets:
        #json.dumps(x)
        script = "INSERT INTO data_tweets (ID, tweet , topic, city_searched, country_searched) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (ID) DO NOTHING;;"
        cursor.execute(script,(tweet['id'],json.dumps(tweet),topic, location.raw['address']['city'], location.raw['address']['country'] ))
        connection.commit()


In [16]:
location = geolocator.reverse('-12.0448728,-77.0488376')
location.raw['address']['city'],location.raw['address']['country']

('Lima', 'Peru')

In [20]:
cursor,connection = connect_db(cred.database_user,cred.database_pass,"192.168.0.22",'5432','data_tweets')

You are connected to -  ('PostgreSQL 12.4 (Ubuntu 12.4-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-10ubuntu2) 9.3.0, 64-bit',) 



In [22]:
geo_data = cred.geo_data
topico_buscar_a ='trump'
topico_buscar_b ='biden'
count = 8000
print(geo_data)

['-33.4672832,-70.652568', '-34.6025287,-58.3796486', '-23.5861315,-46.6582803', '-34.8207362,-56.3765221', '-31.3992876,-64.2643839', '-32.8832979,-68.8760287', '-31.6145747,-60.6980372', '7.7721289,-72.226306', '10.2241386,-67.3411383', '10.4686988,-67.0304528', '4.657911,-74.0959651', '-12.0448728,-77.0488376']


In [28]:
tweets_a= search_by_geo(geo_data, topico_buscar_a, count, cursor, connection  )

Inicia..
Santiago Chile  topic = trump,  Count = 10
Inicia..
Buenos Aires Argentina  topic = trump,  Count = 10
Inicia..
São Paulo Brasil  topic = trump,  Count = 10
Inicia..
Montevideo Uruguay  topic = trump,  Count = 10
Inicia..
Córdoba Argentina  topic = trump,  Count = 10
Inicia..
Ciudad de Mendoza Argentina  topic = trump,  Count = 10
Inicia..
Santa Fe Argentina  topic = trump,  Count = 10
Inicia..
San Cristóbal Venezuela  topic = trump,  Count = 10
Inicia..
La Victoria Venezuela  topic = trump,  Count = 10
Inicia..
Caracas Venezuela  topic = trump,  Count = 10
Inicia..
Bogotá Colombia  topic = trump,  Count = 10


In [29]:
tweets_b= search_by_geo(geo_data, topico_buscar_b, count, cursor, connection  )


Inicia..
Santiago Chile  topic = biden,  Count = 10
Inicia..
Buenos Aires Argentina  topic = biden,  Count = 10
Inicia..
São Paulo Brasil  topic = biden,  Count = 10
Inicia..
Montevideo Uruguay  topic = biden,  Count = 10
Inicia..
Córdoba Argentina  topic = biden,  Count = 10
Inicia..
Ciudad de Mendoza Argentina  topic = biden,  Count = 10
Inicia..
Santa Fe Argentina  topic = biden,  Count = 10
Inicia..
San Cristóbal Venezuela  topic = biden,  Count = 10
Inicia..
La Victoria Venezuela  topic = biden,  Count = 10
Inicia..
Caracas Venezuela  topic = biden,  Count = 10
Inicia..
Bogotá Colombia  topic = biden,  Count = 10
